In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import emcee
import corner
from multiprocessing import Pool
import datetime

In [63]:
# Dados começam 06/03/2020
# Vacinação começa 19/01/2022

Psi = pd.read_csv('../dados_usados/psi.csv')

dados = pd.read_csv("../dados_usados/dados").values

dados_datas = dados[:,1][:-1]
dados_casos = dados[:,2][:-1]
dados_mortes = dados[:,3][:-1]
dados_hospitalizados = dados[:,4][:-1]
dados_hospitalizados_vacinados = dados[:,5][:-1]
dados_infectados_vacinados = dados[:,6][:-1]
dados_vacinados_acumulados = dados[:,7][:-1]

In [50]:
# Facendo média móvel do PSI
copy = Psi.copy()
Psi = np.append(Psi, copy)
Psi = Psi

npd = len(Psi)
Nfo = 8

Psif = [[0]]*npd

for k in range(1, npd):
    if (k-Nfo) < 0 :
        Psif[k] = 1/k*sum(Psi[0:k])
    else:
        Psif[k] = 1/Nfo*sum(Psi[k-Nfo:k])

Psif.pop(0)

[0]

In [51]:
# evitando valores negativos nas vacinações diárias

vacinados_dados_aux =  np.copy(dados_vacinados_acumulados)
# Diffs menores que 0 serão substituidos pela repetição do valor anterior, tornando todo valor igual ou maior que o anterior
for i in range(1,len(vacinados_dados_aux)):
    diff = vacinados_dados_aux[i] - vacinados_dados_aux[i-1]
    if diff < 0:
        vacinados_dados_aux[i-1] = vacinados_dados_aux[i]

novos_vacinados =  np.diff(vacinados_dados_aux)
novos_vacinados = novos_vacinados.tolist()
novos_vacinados.insert(0, 0)

In [52]:
# Parametros do modelo
# Ultima revisão: 05/01/2023
k = 1/4
gamma_a = 1/3.5
gamma_s = 1/4
gamma_h = 0.18
gamma_u = 0.13
mi_u = 0.4
qsi = 0.53 
h = 0.06
h_v = 0.012
mi_h = 0.15
ome_h = 0.14
ome_u = 0.29
delta = 0.31
p = 0.2

N=14136417

In [53]:
# define beta em função do tempo
def define_beta(t0, t1, t, b0, b1):
    if t<20.178:
        return 2.1317
    elif (t>=20.178-8) and (t < 72.94):
        return 1.7645
    elif (t>=72.94) and (t < 148):
        return 1.1281
    elif (t>=107.76354915) and (t < 149.34736536):
        return 2.59807746
    elif(t >= 235.35360579 and t < t0):
        return 1.26315674
    elif (t >= t0 and t < t1):
        return b0
    elif (t >= t1):
        return b1
    else:
        return 1

In [54]:
# define parametros em função do inicio da vacinação
def parametros_vacina(t):
    if t >= 321: # 321 é quando começa a vacina
        delta_av = 0.31
        delta_sv = 0.31
        phi_e = 0.7
        k_v=1/3
        p_v = 0.1
        gamma_av = 1/3.5
        gamma_sv = 1/4
        gamma_vu=0.26
        qsi_v = 0.99
        
        eps=.75

        mi_vh=0.03
        mi_vu=0.08
        gamma_vh=0.36
    else:
        delta_av = 0
        delta_sv = 0
        phi_e = 0
        k_v=0
        p_v = 0
        gamma_av = 0
        gamma_sv = 0
        gamma_vu=0
        qsi_v = 0
        eps=0
        mi_vh=0
        mi_vu=0
        gamma_vh=0

    return (delta_av, delta_sv, phi_e, k_v, p_v, gamma_av, gamma_sv, gamma_vu, qsi_v, eps, mi_vh, mi_vu, gamma_vh)

In [55]:

y1 = np.r_[np.diff(dados_casos), np.diff(dados_mortes), np.diff(dados_infectados_vacinados)]

In [56]:
# condições iniciais

D0          = 0
N0          = 14136417
R0          = 0
H0          = 0
HV0         = 0
U0          = 0
UV0         = 0
V0          = 0
Rv0         = 0
Is0         = 2.015439771376298e-06
Ia0         = 1.8028646508967777e-06
Iav0        = 1.8028646508967777e-06 
Isv0        = 1.8028646508967777e-06 
E0          = 1.7639153732952095e-06
Ev0         = 1.7639153732952095e-06
S0          = (1-Is0-Ia0-E0)
Sv0         = 0 
Nw0         = 0
NwV0        = 0


condicoes_iniciais = [S0,E0,V0, Sv0, Ev0, Ia0, Is0, Iav0, Isv0, H0, HV0, U0, UV0, R0, Rv0, D0, Nw0, NwV0, 0, 0, 0]

In [57]:
# def modelo 
def SEIIHURD(t, y, args):

    # t0, b0, t1, b1, t2, b2, t3, b3, t4, b4, t5, b5, b_else = args

    b0, b1, t0, t1 = args

    beta = define_beta(t0, t1, t, b0, b1)

    beta_v = beta

    tau = novos_vacinados[int(t)]

    delta_av, delta_sv, phi_e, k_v, p_v, gamma_av, gamma_sv, gamma_vu, qsi_v, eps, mi_vh, mi_vu, gamma_vh = parametros_vacina(t)

    N=14873064
    
    psi = Psif[int(t)]
    S=y[0]
    E=y[1]
    V=y[2]
    Sv=y[3]
    Ev=y[4]
    Ia=y[5]
    Is=y[6]
    Iav=y[7]
    Isv=y[8]
    H=y[9]
    Hv=y[10]
    U=y[11]
    Uv=y[12]
    R=y[13]
    Rv=y[14]
    D=y[15]
    Nw=y[16]
    NwV=y[17]
    NHn=y[18]
    NHnV=y[19]
    NIav=y[20]

    dSdt = (-(1-psi)*beta*S*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv)) - tau/N
    dEdt = ((1-psi)*beta*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv)*(S+V)) - k*E
    dVdt = tau/N - phi_e*V - ((1-psi)*beta*V*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv))
    dSvdt =  phi_e*(1-eps)*V - ((1-psi)*beta_v*Sv*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv)) #- phi_e*eps*Sv
    dEvdt = ((1-psi)*beta_v*Sv*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv)) - k_v*Ev
    dIadt = (1-p)*k*E - gamma_a*Ia
    dIsdt = p*k*E - gamma_s*Is
    dIavdt = (1-p_v)*k_v*Ev - gamma_av*Iav
    dIsvdt = p_v*k_v*Ev - gamma_sv*Isv
    dHdt = h*qsi*gamma_s*Is + (1-mi_u+ome_u*mi_u)*gamma_u*U - gamma_h*H   
    dHvdt = h_v*qsi_v*gamma_sv*Isv + (1-mi_vu+ome_u*mi_vu)*gamma_vu*Uv - gamma_vh*Hv    
    dUdt = h*(1-qsi)*gamma_s*Is + ome_h*gamma_h*H - gamma_u*U
    dUvdt = h_v*(1-qsi_v)*gamma_sv*Isv + ome_h*gamma_vh*Hv - gamma_u*Uv
    dRdt = gamma_a*Ia + (1-h)*gamma_s*Is + (1-mi_h)*(1-ome_h)*gamma_h*H
    dRvdt = gamma_av*Iav + (1-h_v)*gamma_sv*Isv + (1-mi_vh)*(1-ome_h)*(gamma_vh*Hv) + phi_e*eps*V
    dDdt = (1-ome_h)*(mi_h*gamma_h*H +mi_vh*gamma_vh*Hv) + (1-ome_u)*(mi_u*gamma_u*U+mi_vu*gamma_vu*Uv)
    dNwdt = p*k*E 
    dNwVdt = tau/N
    dHndt = h*qsi*gamma_s*Is
    dHnvdt = h_v*qsi_v*gamma_sv*Isv 
    dNIAvdt = p_v*k_v*Ev

    return [dSdt, dEdt, dVdt, dSvdt, dEvdt, dIadt, dIsdt, dIavdt, dIsvdt, dHdt, dHvdt, dUdt, dUvdt, dRdt, 
            dRvdt, dDdt, dNwdt, dNwVdt, dHndt, dHnvdt, dNIAvdt]

In [58]:
def model(theta):
    b0, b1, t0, t1 = theta
    params = b0, b1, t0, t1
    sol = solve_ivp(fun = SEIIHURD, t_span = [0, len(novos_vacinados)-1], args=(params,),y0 = condicoes_iniciais, t_eval=np.arange(len(novos_vacinados)-1))
    novos_casos = np.diff(sol.y[-5,:]).astype(float)
    novos_mortos = np.diff(sol.y[-6,:]).astype(float)
    modelo_hospitalizados = np.diff(sol.y[-3,:]).astype(float)
    modelo_hospitalizados_vacinados = np.diff(sol.y[-2,:]).astype(float)
    infectados_vacinados = np.diff(sol.y[-1,:]).astype(float)
    return np.r_[novos_casos*N, novos_mortos*N, infectados_vacinados*N]
def lnlike(theta):
    model_dados = model(theta)
    # errMortos = (novos_mortos - np.diff(dados_mortes)[:len(novos_mortos)]) / np.sqrt(novos_mortos+1)
    # errCasos = (novos_casos - np.diff(dados_casos)[:len(novos_casos)]) / np.sqrt(novos_casos+1)
    # errHosp = (modelo_hospitalizados - np.diff(dados_hospitalizados)[:len(modelo_hospitalizados)]) / np.sqrt(modelo_hospitalizados+1)
    # errHospVac = (modelo_hospitalizados_vacinados - np.diff(dados_hospitalizados_vacinados)[:len(modelo_hospitalizados_vacinados)]) / np.sqrt(modelo_hospitalizados_vacinados+1)
    # errInfecVac = (infectados_vacinados - np.diff(dados_infectados_vacinados)[:len(infectados_vacinados)]) / np.sqrt(infectados_vacinados+1)
    
    # errors = np.r_[errMortos, errCasos, errHosp, errHospVac, errInfecVac] 
    # return errors
    return -0.5 * np.sum(((y1 - model_dados)/0.05) ** 2)

def lnprior(theta):
    b0, b1, t0, t1 = theta
    if 0 < b0 <5 and 0 < b1 < 5 and 234 < t0 < 400 and 400 < t1 < 629:
        return 0.0
    return -np.inf

def lnprob(theta):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf

    likelihood = lp + lnlike(theta)
    return likelihood

def main(p0,nwalkers,niter,ndim,lnprob,data):
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)

    print("Running burn-in...")
    p0, _, _ = sampler.run_mcmc(p0, 100)
    sampler.reset()

    print("Running production...")
    pos, prob, state = sampler.run_mcmc(p0, niter,progress=True)

    return sampler, pos, prob, state

In [59]:
nwalkers = 500; niter = 2500

In [65]:
#Intervalo das condições iniciais para o cenário inicial

intervals= np.array(   [ [0, 5], #b0
                         [0, 5], #b1
                         [235, 400], #t1
                         [400, 629], #t2
                       ])
par0 = np.random.rand(len(intervals))
initial = intervals[:,0] + par0 * (intervals[:,1] - intervals[:,0])
ndim = len(initial)
p0 = [np.array(initial) + 1e-7 * np.random.randn(ndim) for i in range(nwalkers)]

with Pool() as pool:
  sampler1, pos1, prob1, state1 = main(p0,nwalkers,niter,ndim,lnprob,y1)
samples1 = sampler1.flatchain
theta_max1  = samples1[np.argmax(sampler1.flatlnprobability)]
b0, b1, t0, t1 = theta_max1
params = b0, b1, t0, t1
sol1 = solve_ivp(fun = SEIIHURD, t_span = [0, len(novos_vacinados)-1], args=(params,),y0=condicoes_iniciais, t_eval=np.arange(len(novos_vacinados)-1))

Running burn-in...
emcee: Exception while calling your likelihood function:
  params: [3.34298945e+00 4.59207022e-01 2.37633217e+02 6.05460822e+02]
  args: []
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/home/filipecruz/.local/lib/python3.8/site-packages/emcee/ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/tmp/ipykernel_51490/1306637021.py", line 34, in lnprob
    likelihood = lp + lnlike(theta)
  File "/tmp/ipykernel_51490/1306637021.py", line 21, in lnlike
    return -0.5 * np.sum(((y1 - model_dados)/0.05) ** 2)
ValueError: operands could not be broadcast together with shapes (1833,) (1830,) 


ValueError: operands could not be broadcast together with shapes (1833,) (1830,) 

In [ ]:
# plotando mortes
ts0 =  np.arange(len(novos_vacinados)-1)

qi = sol1.y[-5,:] #mortes acumulados

#Criando plot
fig, ax = plt.subplots(figsize=(16,8))

#Desenhando grid no plot
ax.grid(which='major', axis='both', color='black',linewidth=1.,alpha=0.3)
ax.autoscale()

x_para_escala = np.arange(0, len(dados_datas)-1, 1)

ax.scatter(x_para_escala,  np.diff(dados_casos), color='black',zorder=3,label='Reported data')
#Plot the fitted function as a line.
ax.plot(np.diff(N*qi),color='blue',label='Fitted function')
#Set the labels
ax.set_ylabel('Casos',fontsize=22)
ax.set_xlabel('Dias', fontsize=22)

# plt.vlines([ 333.15303923, 439.95810453], ymin=0, ymax=200)
#Set the title
#The size of the numbers on the axixis
ax.tick_params(labelsize=18)

#Limiting the ammount of dates on the X axixs
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
#Rotating the dates for better visualization
# plt.setp(ax.get_xticklabels(), rotation=30)

#Show Graph       
plt.show()

In [ ]:
# plotando mortes
ts0 =  np.arange(len(novos_vacinados)-1)

qi = sol1.y[-6,:] #mortes acumulados

#Criando plot
fig, ax = plt.subplots(figsize=(16,8))

#Desenhando grid no plot
ax.grid(which='major', axis='both', color='black',linewidth=1.,alpha=0.3)
ax.autoscale()

x_para_escala = np.arange(0, len(dados_datas)-1, 1)

ax.scatter(x_para_escala,  np.diff(dados_mortes), color='black',zorder=3,label='Reported data')
#Plot the fitted function as a line.
ax.plot(np.diff(N*qi),color='blue',label='Fitted function')
#Set the labels
ax.set_ylabel('Mortos',fontsize=22)
ax.set_xlabel('Dias', fontsize=22)

# plt.vlines([ 333.15303923, 439.95810453], ymin=0, ymax=200)
#Set the title
#The size of the numbers on the axixis
ax.tick_params(labelsize=18)

#Limiting the ammount of dates on the X axixs
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
#Rotating the dates for better visualization
# plt.setp(ax.get_xticklabels(), rotation=30)

#Show Graph       
plt.show()

In [ ]:
# plotando mortes
ts0 =  np.arange(len(novos_vacinados)-1)

qi = sol1.y[-3,:] #mortes acumulados

#Criando plot
fig, ax = plt.subplots(figsize=(16,8))

#Desenhando grid no plot
ax.grid(which='major', axis='both', color='black',linewidth=1.,alpha=0.3)
ax.autoscale()

x_para_escala = np.arange(0, len(dados_datas)-1, 1)

ax.scatter(x_para_escala,  np.diff(dados_hospitalizados), color='black',zorder=3,label='Reported data', s=2)
#Plot the fitted function as a line.
ax.plot(np.diff(N*qi),color='blue',label='Fitted function')
#Set the labels
ax.set_ylabel('Hospitalizados',fontsize=22)
ax.set_xlabel('Dias', fontsize=22)

# plt.vlines([ 333.15303923, 439.95810453], ymin=0, ymax=200)
#Set the title
#The size of the numbers on the axixis
ax.tick_params(labelsize=18)

#Limiting the ammount of dates on the X axixs
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
#Rotating the dates for better visualization
# plt.setp(ax.get_xticklabels(), rotation=30)

#Show Graph       
plt.show()

In [ ]:
# plotando mortes
ts0 =  np.arange(len(novos_vacinados)-1)

qi = sol1.y[-2,:] #mortes acumulados

#Criando plot
fig, ax = plt.subplots(figsize=(16,8))

#Desenhando grid no plot
ax.grid(which='major', axis='both', color='black',linewidth=1.,alpha=0.3)
ax.autoscale()

x_para_escala = np.arange(0, len(dados_datas)-1, 1)

ax.scatter(x_para_escala,  np.diff(dados_hospitalizados_vacinados), color='black',zorder=3,label='Reported data', s=2)
#Plot the fitted function as a line.
ax.plot(np.diff(N*qi),color='blue',label='Fitted function')
#Set the labels
ax.set_ylabel('Hospitalizados Vacinados',fontsize=22)
ax.set_xlabel('Dias', fontsize=22)

# plt.vlines([ 333.15303923, 439.95810453], ymin=0, ymax=200)
#Set the title
#The size of the numbers on the axixis
ax.tick_params(labelsize=18)

#Limiting the ammount of dates on the X axixs
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
#Rotating the dates for better visualization
# plt.setp(ax.get_xticklabels(), rotation=30)

#Show Graph       
plt.show()

In [ ]:
# plotando mortes
ts0 =  np.arange(len(novos_vacinados)-1)

qi = sol1.y[-1,:] #mortes acumulados

#Criando plot
fig, ax = plt.subplots(figsize=(16,8))

#Desenhando grid no plot
ax.grid(which='major', axis='both', color='black',linewidth=1.,alpha=0.3)
ax.autoscale()

x_para_escala = np.arange(0, len(dados_datas)-1, 1)

ax.scatter(x_para_escala,  np.diff(dados_infectados_vacinados), color='black',zorder=3,label='Reported data', s=2)
#Plot the fitted function as a line.
ax.plot(np.diff(N*qi),color='blue',label='Fitted function')
#Set the labels
ax.set_ylabel('Casos Vacinados',fontsize=22)
ax.set_xlabel('Dias', fontsize=22)

# plt.vlines([ 333.15303923, 439.95810453], ymin=0, ymax=200)
#Set the title
#The size of the numbers on the axixis
ax.tick_params(labelsize=18)

#Limiting the ammount of dates on the X axixs
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
#Rotating the dates for better visualization
# plt.setp(ax.get_xticklabels(), rotation=30)

#Show Graph       
plt.show()

In [ ]:
import corner
flat_samples = sampler1.get_chain(discard=100, thin=15, flat=True)
b0, b1, t0, t1 = theta_max1
print(flat_samples.shape)
labels = ["b0","b1","t0","t1",]
fig = corner.corner(
    flat_samples, labels=labels, truths=[b0, b1, t0, t1]
);